In [161]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re
import sys

sys.path.append(os.path.abspath("../"))
from dataset import DS_NAMES
from dataset import DS_DIMENSIONS
from utilities.figure_tools import get_latest_results_directory, make_fig_directory, load_results

cmap = plt.get_cmap("Set1")
colors = cmap.colors

results_path = "../results/final/compare_methods.csv"
output_dir = "./compare-methods/"
export_figures = False

if export_figures and not os.path.isdir(output_dir):
    os.makedirs(output_dir)

In [162]:
all_results = pd.read_csv(results_path)
all_results = all_results[all_results["n_trees"] == 10]
all_results = all_results.groupby(["dataset", "explainer"]).mean().reset_index()
print("explainers:", list(all_results["explainer"].unique()))
print("datasets", list(all_results["dataset"].unique()))
all_results.head(100)

explainers: ['AFT', 'FACETIndex', 'MACE', 'OCEAN', 'RFOCSE']
datasets ['cancer', 'glass', 'magic', 'spambase', 'vertebral']


,dataset,explainer,n_trees,max_depth,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain
0,cancer,AFT,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,0.370,0.194152,1.860480,0.000002,0.036073,0.001804,20.0
1,cancer,FACETIndex,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.000,0.153822,4.325000,4.843777,0.409259,0.020463,20.0
2,cancer,MACE,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.000,1.365519,30.000000,0.004707,42.294541,2.114727,20.0
3,cancer,OCEAN,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.000,0.156526,11.410000,0.192969,18.280266,0.914013,20.0
4,cancer,RFOCSE,10.0,5.0,0.0,0.938596,0.950000,0.883721,0.915663,1.000,0.164461,4.650000,0.024839,40020.335390,2001.016770,20.0
5,glass,AFT,10.0,5.0,4.5,0.821212,0.835234,0.861628,0.843633,1.000,0.085238,1.485000,0.000001,0.034907,0.001745,20.0
6,glass,FACETIndex,10.0,5.0,4.5,0.821212,0.835234,0.861628,0.843633,1.000,0.085770,2.745000,2.523329,0.020992,0.001050,20.0
7,glass,MACE,10.0,5.0,4.5,0.821212,0.835234,0.861628,0.843633,0.990,0.753847,8.969737,0.002929,215.083105,10.754155,20.0
8,glass,OCEAN,10.0,5.0,4.5,0.821212,0.835234,0.861628,0.843633,1.000,0.081336,4.290000,0.185990,27.752488,1.387624,20.0
9,glass,RFOCSE,10.0,5.0,4.5,0.821212,0.835234,0.861628,0.843633,1.000,0.055554,1.885000,0.008334,56.661614,2.833081,20.0


In [163]:
# facet_idx = (all_results["explainer"] == "FACETIndex")
# all_results[facet_idx]["avg_dist"].head()
# all_results["norm_dist"] = all_results["avg_dist"] / all_results[facet_idx]["avg_dist"]
all_results.head()

,dataset,explainer,n_trees,max_depth,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain
0,cancer,AFT,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,0.37,0.194152,1.86048,0.000002,0.036073,0.001804,20.0
1,cancer,FACETIndex,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,0.153822,4.32500,4.843777,0.409259,0.020463,20.0
2,cancer,MACE,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,1.365519,30.00000,0.004707,42.294541,2.114727,20.0
3,cancer,OCEAN,10.0,5.0,4.5,0.946491,0.940244,0.908325,0.923360,1.00,0.156526,11.41000,0.192969,18.280266,0.914013,20.0
4,cancer,RFOCSE,10.0,5.0,0.0,0.938596,0.950000,0.883721,0.915663,1.00,0.164461,4.65000,0.024839,40020.335390,2001.016770,20.0


In [166]:
# df = all_results.copy() #.set_index(["explainer", "dataset"])
# df.reset_index()
# for expl in all_results["explainer"].unique():
#     for ds in all_results["dataset"].unique():
#         facet_idx = (df["explainer"] == "FACETIndex") & (df["dataset"] == ds)
#         expl_idx = (df["explainer"] == expl) & (df["dataset"] == ds)    
#         df.loc[expl_idx, "avg_dist"] = float(df.loc[expl_idx, "avg_dist"]) / float(df.loc[facet_idx, "avg_dist"])
        # print()
        # print(df.iloc[facet_idx])
        # print(df.loc[[expl, ds]]["avg_dist"] / df.loc[["FACETIndex", ds]]["avg_dist"])
        # print(df.loc[expl, ds]["norm_dist"])
        # df.loc[expl, "avg_dist"] = df.loc[expl, "avg_dist"].div(df.loc["FACETIndex", "avg_dist"])
    # print()
# df.head(100)
# df[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["avg_dist"])

avg_dist                                         
explainer       AFT FACETIndex      MACE     OCEAN    RFOCSE
dataset                                                     
cancer     1.262186        1.0  1.365519  0.156526  0.164461
glass      0.993791        1.0  0.753847  0.081336  0.055554
magic      1.237344        1.0  0.728178  0.100905  0.075638
spambase   3.031838        1.0  3.042850  0.005249  0.037012
vertebral  1.217404        1.0  0.454783  0.072163  0.059878

In [153]:
# facet_dists.head()
norm_data = all_results.copy()
for expl in norm_data["explainer"].unique():
    for ds in norm_data["dataset"].unique():
        facet_idx = (all_results["explainer"] == "FACETIndex") & (all_results["dataset"] == ds)
#         facet_dist = all_results[facet_idx]["avg_dist"]

#         match_idx = (all_results["explainer"] == "FACETIndex") & (all_results["dataset"] == ds)
#         norm_data[match_idx] = norm_data[match_idx] / facet_dist

# norm_data["avg_dist"] = norm_data["avg_dist"] / facet_dists
# norm_data.head()

In [154]:
df = all_results.set_index(["dataset", "explainer"])
metrics = ["sample_time", "avg_dist", "per_valid"]

with open("compare_methods_table.csv", "w") as f:
    # header row one
    f.write("dataset,")
    for expl in all_results["explainer"].unique():
        for m in metrics:
            f.write(expl + ",")
    f.write("\n")
    # header row two
    f.write(",")
    for expl in all_results["explainer"].unique():
        for m in metrics:
            f.write(m + ",")
    f.write("\n")
    # body row
    for ds in all_results["dataset"].unique():
        f.write(ds + ",")
        for expl in all_results["explainer"].unique():
            for m in metrics:
                f.write(str(df.loc[ds, expl][m]) + ",")
        f.write("\n")

In [155]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["per_valid"])

per_valid                               
explainer       AFT FACETIndex   MACE OCEAN RFOCSE
dataset                                           
cancer         0.37        1.0  1.000   1.0  1.000
glass          1.00        1.0  0.990   1.0  1.000
magic          0.99        1.0  1.000   1.0  0.995
spambase       0.76        1.0  1.000   1.0  1.000
vertebral      1.00        1.0  0.995   1.0  1.000

In [156]:
all_results[""]

KeyError: ''

In [ ]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["avg_dist"])

avg_dist                                         
explainer       AFT FACETIndex      MACE     OCEAN    RFOCSE
dataset                                                     
cancer     0.194152   0.153822  1.365519  0.156526  0.164461
glass      0.085238   0.085770  0.753847  0.081336  0.055554
magic      0.118433   0.095715  0.728178  0.100905  0.075638
spambase   0.024882   0.008207  3.042850  0.005249  0.037012
vertebral  0.080469   0.066099  0.454783  0.072163  0.059878

In [ ]:
all_results[["dataset", "explainer", "avg_dist", "per_valid", "prep_time", "sample_time"]].pivot(index=["dataset"], columns=["explainer"], values=["sample_time"])

sample_time                                             
explainer         AFT FACETIndex       MACE     OCEAN       RFOCSE
dataset                                                           
cancer       0.001804   0.020463   2.114727  0.914013  2001.016770
glass        0.001745   0.001050  10.754155  1.387624     2.833081
magic        0.003289   0.001736   2.267405  2.246202    70.124059
spambase     0.002600   0.015470  20.438034  0.585131  2249.782774
vertebral    0.001922   0.001436   3.858280  1.214346     4.099172

In [ ]:
all_results = pd.read_csv(results_path)
all_results = all_results[all_results["n_trees"] == 100]
# all_results["diff"] = all_results["sample_time"] - (((all_results["sample_time"] * all_results["n_explain"]) - ((1 - all_results["per_valid"]) * all_results["n_explain"] * 300)) / all_results["n_explain"])
tot_time = all_results["sample_time"] * all_results["n_explain"]
n_timeout = all_results["per_valid"] * all_results["n_explain"]
max_time = 300
timeout_cost = n_timeout * max_time
all_results["adj_time"] = (tot_time - timeout_cost) / all_results["n_explain"]
# all_results = all_results.groupby(["dataset", "explainer"]).mean().reset_index()
# print("explainers:", list(all_results["explainer"].unique()))
# print("datasets", list(all_results["dataset"].unique()))
all_results.head()

,dataset,explainer,n_trees,max_depth,iteration,accuracy,precision,recall,f1,per_valid,avg_dist,avg_length,prep_time,explain_time,sample_time,n_explain,adj_time
